<a href="https://colab.research.google.com/github/AISushilRaj/Zeotap/blob/main/SUSHILRAJ_GADDAM_Task_2_Lookalike_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import io
import pandas as pd
from google.colab import files

cuploaded = files.upload()
uploaded_filename = list(cuploaded.keys())[0]
customerpreview = pd.read_excel(io.BytesIO(cuploaded[uploaded_filename]))

Saving CustomersXL.xls to CustomersXL.xls


In [2]:
tuploaded = files.upload()
tuploaded_filename = list(tuploaded.keys())[0]
Transactionspreview = pd.read_excel(io.BytesIO(tuploaded[tuploaded_filename]))

Saving TransactionsXL.xls to TransactionsXL.xls


In [4]:
puploaded = files.upload()
puploaded_filename = list(puploaded.keys())[0]
productsspreview = pd.read_excel(io.BytesIO(puploaded[puploaded_filename]))

Saving ProductsXL.xls to ProductsXL.xls


In [5]:
transactions_merged = Transactionspreview.merge(customerpreview, on="CustomerID", how="left").merge(productsspreview, on="ProductID", how="left")


In [6]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Assuming you have DataFrames: customerpreview, productspreview, Transactionspreview, transactions_merged

# Focus on CustomerID: C0001 - C0020
target_customers = customerpreview[customerpreview['CustomerID'].isin([f'C{i:04d}' for i in range(1, 21)])]

# Feature Engineering: Aggregating customer-level data
customer_features = transactions_merged.groupby("CustomerID").agg(
    total_spending=("TotalValue", "sum"),
    purchase_frequency=("TransactionID", "count"),
    avg_quantity=("Quantity", "mean"),
    category_preference=("Category", lambda x: x.mode()[0] if not x.mode().empty else None),
    region=("Region", "first")
).reset_index()

# One-Hot Encoding categorical columns
encoder = OneHotEncoder(sparse_output=False)  # sparse=False for dense output
encoded_features = encoder.fit_transform(customer_features[['category_preference', 'region']])

# Combine numerical and encoded features
numerical_features = customer_features[["total_spending", "purchase_frequency", "avg_quantity"]].values
customer_vectors = np.hstack([numerical_features, encoded_features])

# Compute Cosine Similarity for all customers
similarity_matrix = cosine_similarity(customer_vectors)

# Generate recommendations for the first 20 customers
lookalike_map = {}

for i, customer_id in enumerate(target_customers["CustomerID"]):
    similarity_scores = similarity_matrix[i]
    top_indices = np.argsort(similarity_scores)[-4:-1][::-1]  # Get top 3 excluding self
    lookalike_map[customer_id] = [
        (customer_features.iloc[idx]["CustomerID"], similarity_scores[idx])
        for idx in top_indices
    ]

# Print the results
for customer_id, lookalikes in lookalike_map.items():
    print(f"Top 3 Lookalikes for Customer {customer_id}:")
    for lookalike, similarity in lookalikes:
        print(f"- {lookalike} (Similarity: {similarity:.4f})")
    print("\n")

Top 3 Lookalikes for Customer C0001:
- C0039 (Similarity: 1.0000)
- C0048 (Similarity: 1.0000)
- C0190 (Similarity: 1.0000)


Top 3 Lookalikes for Customer C0002:
- C0134 (Similarity: 1.0000)
- C0088 (Similarity: 1.0000)
- C0106 (Similarity: 1.0000)


Top 3 Lookalikes for Customer C0003:
- C0052 (Similarity: 1.0000)
- C0031 (Similarity: 1.0000)
- C0129 (Similarity: 1.0000)


Top 3 Lookalikes for Customer C0004:
- C0165 (Similarity: 1.0000)
- C0126 (Similarity: 1.0000)
- C0137 (Similarity: 1.0000)


Top 3 Lookalikes for Customer C0005:
- C0186 (Similarity: 1.0000)
- C0007 (Similarity: 1.0000)
- C0146 (Similarity: 1.0000)


Top 3 Lookalikes for Customer C0006:
- C0171 (Similarity: 1.0000)
- C0168 (Similarity: 1.0000)
- C0187 (Similarity: 1.0000)


Top 3 Lookalikes for Customer C0007:
- C0115 (Similarity: 1.0000)
- C0005 (Similarity: 1.0000)
- C0140 (Similarity: 1.0000)


Top 3 Lookalikes for Customer C0008:
- C0139 (Similarity: 1.0000)
- C0098 (Similarity: 1.0000)
- C0109 (Similarity: 1.

In [13]:
import pandas as pd

lookalike_df = pd.DataFrame(columns=['CustomerID', 'S1', 'S2', 'S3', 'Similarity1', 'Similarity2', 'Similarity3'])

for customer_id, lookalikes in lookalike_map.items():
    row = {'CustomerID': customer_id}
    for i, (lookalike, similarity) in enumerate(lookalikes):
        row[f'S{i+1}'] = lookalike
        row[f'Similarity{i+1}'] = similarity
    lookalike_df = pd.concat([lookalike_df, pd.DataFrame([row])], ignore_index=True)

# Download to Excel
from google.colab import files
lookalike_df.to_excel("lookalikes_restructured.xlsx", index=False)
files.download("lookalikes_restructured.xlsx")

<ipython-input-13-eaf8d5f84dd1>:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  lookalike_df = pd.concat([lookalike_df, pd.DataFrame([row])], ignore_index=True)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>